# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [28]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [29]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts


## Define machine and code 

In [50]:
facility='cori'
img_size=64
run_type='3dcgan'
assert facility in ['cori','summit'], "Error%s"%(facility)
assert img_size in [64,128,512]
assert run_type in ['2dgan','2dcgan','3dgan','3dcgan']

## Read ref_launch.yaml and define dictionary

In [51]:
start_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts'
launch_cfile=start_dir+'/ref_launch.yaml'

with open(launch_cfile) as f:
    config_dict= yaml.load(f, Loader=yaml.SafeLoader)

In [52]:
## Read ref_launch.yaml
main_code_dir=config_dict[facility]['main_code_dir']
dict_pars=config_dict[facility][img_size][run_type]
dict_pars.update({'nodes':1,'job_name':'analysis_'+run_type,
                 'queue':'debug','bins':'unenven','cores':64,'time':'00:30:00'})

dict_pars['code_dir']=main_code_dir+dict_pars['code_dir']
dict_pars['staging_loc']=config_dict[facility]['staging_loc']
print(dict_pars)
staging_loc=dict_pars['staging_loc']

{'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code', 'config': 'config_3d_cgan.yaml', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/', 'nodes': 1, 'job_name': 'analysis_3dcgan', 'queue': 'debug', 'bins': 'unenven', 'cores': 64, 'time': '00:30:00', 'staging_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'}


In [53]:
config_dict['cori'][128]

{'2dgan': {'code_dir': 'code/1_basic_GAN/1_main_code/DDP_new_loss',
  'config': 'config_2dgan.yaml',
  'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'},
 '2dcgan': {'code_dir': 'code/3_cond_GAN/1_main_code/DDP',
  'config': 'config_2dgan.yaml',
  'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/'}}

In [54]:
%%bash -s "$staging_loc"
ls $1/3d_cGAN

20210223_210217_3dcgan_predict_0.8_m2
20210227_050213_3dcgan_predict_0.65_m2
20210309_200006_3dcgan_predict_0.65_m2
20210313_050345_3dcgan_predict_0.65_m2
20210417_83132_train


In [55]:
ip_folder='20210417_83132_train'

dict_pars['ip_folder']=dict_pars['staging_loc']+'/3d_cGAN/'+ip_folder

In [56]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir={code_dir}

python $code_dir/post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [57]:
fname='batch_analysis_{0}.sh'.format(run_type)
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis_3dcgan.sh


In [58]:
### Move to staging locations in project space:
# os.chdir(dict_pars['staging_loc'])

In [59]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_3dcgan

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code

python $code_dir/post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results//3d_cGAN/20210417_83132_train  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/ -c 64 -bin unenven
conda deactivate
echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [95]:
%%bash -s "$filename" ## Use python variable in bash
# module load cgpu
sbatch $1

Submitted batch job 41691564
